## Задание 07

### Задача 1. Смещение константного алгоритма (2 балла)
Пусть $x \in R^d$, и значение каждого признака на объекте $x$ независимо генерируется из равномерного распределения $x_i\in U[0,1],i=1,2\dots,d$. Будем считать, что объекты в выборке независимы, а $\mathbb{E}[y|x]=x^Tx.$ Найдите смещение константного алгоритма, полученного минимизацией среднеквадратичной ошибки на обучающей выборке. 

Найдём смещение по формуле 
 $$ bias = E_x[{(E_X[\mu (X)] - E[y|x])}^2] = E_x[{( c - x^T x )}^2] = c^2 - 2 c E_x [x^T x] + E_x[ {(x^T x)}^2 ]$$ , где $c=\mu(X)$ - константный алгоритм.
 
Так как мы имеем стандартное равномерное распределение (плотность = 1), то мат ожидание будет равно:

$$E_x[x^T x] = \int_{{[ 0, 1]}^d} (x_1^2 + ... + x_d^2) dx_1 ... dx_d = \int_0^1((d-1)\frac{1}{3} + x_d^2 ) dx_d = \frac{d}{3}$$

$$E_x[{(x^T x)}^2] = \int_{{[ 0, 1 ]}^d} {(x_1^2 + ... + x_d^2)}^2 d x_1 ... d x_d = \int_{{[ 0, 1]}^d} ( \sum_{i} x_i^4 + \sum_{i \ne j} x_i^2 x_j^2) d x_1 ... d x_d = \int_0^1 ((d-1)\frac{1}{5} + x_d^4 )d x_d + \int_0^1 ((d-1)\frac{1}{3} + x_d^2 ) dx_d \int_0^1 ((d-2)\frac{1}{3} + x_{d-1}^2) dx_{d-1} = \frac{d}{5} + \frac{d(d - 1)}{9}$$

В итоге

$$bias = c^2 - \frac{2cd}{3} + \frac{d}{5} + \frac{d(d - 1)}{9}$$

### Задача 2. Разложение ошибки (2 балла)
Истинная зависимость имеет вид $y_i = 3x^2_i+ u_i$, где $y_i$ - прогнозируемая переменная, $x_i$-признак и $u_i$ - случайная составляющая. Величины $x_i$ независимы и равновероятно принимаю значения $0, 1, 2$. Величины $u_i$ независимы и равновероятно принимают значения $−1$ и $1$.
Исследователь Анатолий оценивает модель линейной регрессии $y_i = wx_i$, минимизируя среднеквадратичную ошибку.
Разложите ожидание квадрата ошибки прогноза на шум, смещение и разброс.

### Задача 3. Взвешенное голосование (2 балла)
Рассмотрим задачу бинарной классификации, пусть у нас есть три алгоритма $b_1(x), b_2(x)$ и $b_3(x)$, каждый из которых ошибается с вероятностью $p$. Мы строим композицию взвешенным голосованием: алгоритмам присвоены значимости $w_1, w_2$ и $w_3$, и для вынесения вердикта суммируются значимости алгоритмов, проголосовавших за каждый из классов:
$$
a_0=\sum_{i=1}^3w_i[b_i(x)=0], $$
$$
a_1=\sum_{i=1}^3w_i[b_i(x)=1]. \\
$$
Объект $x$ относится к классу, для которого сумма оказалась максимальной. Например, если первые два алгоритма голосуют за класс $0$, а третий за класс $1$, то выбирается класс $0$, если $w_1+w_2>w_3$, и класс $1$ в противном случае. Какова вероятность ошибки такой композиции этих трех алгоритмов, если:

* $w_1=0.3, w_2=0.4, w_3=0.3$;
* $w_1=0.2, w_2=0.5, w_3=0.2$?


*1 случай* $w_1=0.3, w_2=0.4, w_3=0.3$

Если мы возьмём любые два веса, их сумма будет больше третьего. Соответетсвенно, правильый ответ мы получим, если как минимум два алгоритма дали верное предсказание. Это возможно, если все три алгоритма ошибутся или два из них. Соответсвенно, вероятность ошибки композиции $$P = p^3 + 3p^2(1-p)$$


*2 случай* $w_1=0.2, w_2=0.5, w_3=0.2$

Вес второго алгоритма больше суммы двух других, соответственно ошибка композиции ложится всецело на второй алгоритм. Соответсвенно,
$$P = p$$

### Задача 4. Разложение ошибки с помощью бутстрапа 

В этом задании вам предстоит воспользоваться возможностями bootstraping для оценки смещения и разброса алгоритмов машинного обучения. Делать мы это будем на данных boston:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()
X = boston["data"]
y = boston["target"]
X.shape, y.shape

((506, 13), (506,))

#### Алгоритм оценки смещения и разброса алгоритма $a$

1. Сгенерировать $s$ выборок $X_j$ методом бутстрапа.

2. На каждой выборке $X_j$ обучить алгоритм $a_j$.

3. Для каждой выборки $X_j$ определить множество объектов $T_j$, не вошедших в нее (out-of-bag). Вычислить предсказания алгоритма $a_j$ на объектах $T_j$.

Поскольку у нас есть только один ответ для каждого объекта, мы будем считать шум равным 0, а $\mathbb{E}[y|x]$ равным имеющемуся правильному ответу для объекта $x$.

#### Итоговые оценки:

* #### Смещение: 
для одного объекта - квадрат разности среднего предсказания и правильного ответа. Среднее предсказание берется только по тем алгоритмам $a_j$, для которых этот объект входил в out-of-bag выборку $T_j$. Для получения общего смещения выполнить усреденение смещений по объектам.
* #### Разброс: 
для одного объекта - выборочная дисперсия предсказаний алгоритмов $a_j$, для которых этот объект входил в out-of-bag выборку $T_j$. Для получения общего разброса выполнить усреденение разбросов по объектам.
* #### Ошибка $L$:
усреднить квадраты разностей предсказания и правильного ответа по всем выполненным предсказаниям для всех объектов.

В результате должно получиться, что ошибка приблизительно равна сумме смещения и разброса!

* реализуйте описанный алгоритм. Обратите внимание, что если объект не вошел ни в одну из out-of-bag выборок, учитывать его в вычислении итоговых величин не нужно. (3 балла) 

* Оцените смещение, разброс и ошибку для трех алгоритмов с гиперпараметрами по умолчанию: линейная регрессия, решающее дерево, случайный лес.

* Проанализируйте полученный результат. Согласуются ли полученные результаты с теми, что мы обсуждали на занятиях? (1 балл)

In [54]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

50

In [129]:
def bootstrap(X, y, alg, s, l):
    bias = 0.0
    var = 0.0
    L = 0.0
    dic = np.ndarray(shape = y.shape[0], dtype=list)
    
    oobi = set()
    
    for i in range(s):
        idx = np.random.randint(0,y.shape[0], l)
        T_idx = [i for i in range(y.shape[0]) if i not in idx]
        oobi |= set(T_idx)
        
        alg.fit(X[idx], y[idx]) 
        y_predict = alg.predict(X[T_idx])    
        for p,i in zip(y_predict, T_idx):
            if(dic[i] == None):
                dic[i] = [p]
            else:
                dic[i].append(p)
    
    co = 0
    for i in oobi:
        bias+= 1.0/len(oobi)*(np.mean(dic[i])-y[i])**2
        
        var += 1.0/(len(oobi)*len(dic[i]))*sum((dic[i]-np.mean(dic[i]))**2)
        
        co+=len(dic[i])
        L += sum((dic[i]-y[i])**2)
    L/=co
    return bias, var, L

print("For Linear regression bias, variance and L are ", bootstrap(X, y, LinearRegression(), 10, 250))
print("For DecisionTreeRegressor bias, variance and L are ",bootstrap(X, y, DecisionTreeRegressor(), 10, 250))
print("For RandomForest bias, variance and L are ",bootstrap(X, y, RandomForestRegressor(), 10, 250))

For Linear regression bias, variance and L are  (24.455470779760645, 2.314336219497826, 26.68801268034769)
For DecisionTreeRegressor bias, variance and L are  (12.73884816030611, 12.75048802545533, 26.0721627756161)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


For RandomForest bias, variance and L are  (12.948132041112343, 3.5715238428828853, 16.594528346709467)


Во всех случаях подвердилось утверждение, что ошибка приблизительно равна сумме смещения и разброса.

Для линейной регрессии имеем большое смещение, так как мы приближаем нелинейную функцию, однако имеем маленький разброс.

Для деревьев имеем более низкий разброс, так как ими можно приближать нелинейные зависимости, но разброс больше, так как они более неустойчивые.

У леса мы получили сопоставимое с обычным деревом смещение, однако раазброс значительно понизился.